In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
import types
owlready2.reasoning.JAVA_MEMORY = 6000

import scipy
#from scipy.spatial import ConvexHull
#import cdd
#from cdd import RepType, Matrix, Polyhedron
#from fractions import Fraction

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import random

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [2]:
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/goslimyeast.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/galennorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/gonorm.xml.owl'
dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/family_ontology.owl'

onto = get_ontology(dir)
onto = onto.load()

individuals = list(onto.individuals())
roles = list(onto.properties())
classes = list(onto.classes())
gcas = list(onto.general_class_axioms())

classes = list(onto.classes())

for cls in classes:
    print(f'{cls} Ancestors: {cls.ancestors(include_self = True, include_constructs = True)}')
    #print(f'{cls} Constructs: {list(cls.constructs())}')
    #print(f'{cls} Properties: {list(cls.get_class_properties())}')
    print('')

with onto:
    sync_reasoner()

classes = list(onto.classes())

for cls in classes:
    print(f'{cls} Ancestors: {cls.ancestors(include_self = True, include_constructs = True)}')
    print(f'{cls} Constructs: {list(cls.constructs())}')
    print(f'{cls} Properties: {list(cls.get_class_properties())}')
    print('')

In [5]:
'''
Class for creating entities to
populate the creation of the
canonical models.

The .name attribute is used to
create a single representation
for concepts like A and B / 
B and A, as they are the same.
'''

class CanonicalModelElements:

    concept_names = {}
    concept_intersections = {}
    concept_restrictions = {}
    all_concepts = {}

    def __init__(self, concept):
        self.concept = concept
        self.name = self.get_name()
        self.get_element_dict()

    def get_name(self):

        # add \Top
        
        if type(self.concept) == ThingClass:
            return self.concept.name

        elif type(self.concept) == Restriction:
            return 'exists_' + self.concept.property.name + '.' + self.concept.value.name
        
        else:
            return 'And_' + ''.join(sorted(self.concept.Classes[0].name + self.concept.Classes[1].name)) # The name is sorted to avoid that (e.g) (A \and B) and (B \and A) are treated as different concepts
        
    def get_element_dict(self):

        if type(self.concept) == ThingClass:
            CanonicalModelElements.concept_names[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == Restriction:
            CanonicalModelElements.concept_restrictions[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == And:
            CanonicalModelElements.concept_intersections[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

In [6]:
def get_canonical_model_elements(concept_names_iter, role_names_iter, ontology):
    
    onto = ontology
    top = owl.Thing
    bottom = owl.Nothing

    CanonicalModelElements(top)
    CanonicalModelElements(bottom)

    for concept_name in concept_names_iter:
        
        CanonicalModelElements(concept_name)
        for concept_name2 in concept_names_iter:
        
            with onto:
                gca = GeneralClassAxiom(concept_name & concept_name2)
                gca.is_a.append(concept_name & concept_name2)
            
            CanonicalModelElements(gca.left_side)

    print('')
    print('===========================================================================================================')
    print('All Concept Names and Concept Intersections have been preprocessed for the creation of the canonical model.')
    print('===========================================================================================================')

    concept_names_iter.append(top)
    # concept_names_iter.append(bottom)

    for role_name in role_names_iter:
        for concept_name in concept_names_iter:
            with onto:
                gca = GeneralClassAxiom(role_name.some(concept_name))
                gca.is_a.append(role_name.some(concept_name))

            CanonicalModelElements(gca.left_side)

    print('')
    print('')
    print('All restrictions have been preprocessed for the creation of the canonical model.')

In [7]:
'''
The main class for creating the canonical model for the ontology.

The canonical model is stored in dictionaries available as class variables 'concept_canonical_interpretation'
and 'role_canonical_interpretation'. 

Args:
    concept_names_dict: a dictionary stored in the CanonicalModelElement class.
    concept_intersection_dict: a dictionary stored in the CanonicalModelElement class.
    concept_restrictions_dict: a dictionary stored in the CanonicalModelElement class.
    all_concepts_dict: a dictionary stored in the CanonicalModelElement class.
    role_names_iter (list): a list containing all role names in the loaded ontology.
'''

class CanonicalModel:

    concept_canonical_interpretation = {}
    role_canonical_interpretation = {}

    def __init__(self, concept_names_dict, concept_intersections_dict, concept_restrictions_dict, all_concepts_dict, role_names_iter):
        
        self.domain = all_concepts_dict
        self.concept_names_dict = concept_names_dict
        self.concept_restrictions_dict = concept_restrictions_dict
        self.concept_intersections_dict = concept_intersections_dict

        self.role_names_iter = role_names_iter

        self.concept_canonical_interp = self.get_concept_name_caninterp() # These are only used to build the concept_canonical_interpretation and role_canonical_interpretation class attributes
        self.role_canonical_interp = self.get_role_name_caninterp()       # The functions do not return anything, they just update their corresponding class variables 

    def get_concept_name_caninterp(self):

        # The variable concept is a string containing the name of an element of the domain of the canonical model
        # The key to the concept_names_dict variable corresponds to concept.name
        # This name can be used to access the concept in owlready2's format

        for concept in self.concept_names_dict.keys():

            CanonicalModel.concept_canonical_interpretation[concept] = []
            superclasses = self.domain[concept].concept.ancestors(include_self=True, include_constructs=True) # The self.domain[concept] is used to access the CanonicalModelElements type of object,
                                                                                                               # and the attribute .concept is used to access the concept in owlready2 format
                                                                                                              
            for superclass in superclasses:

                if type(superclass) == ThingClass:
                    CanonicalModel.concept_canonical_interpretation[concept].append(superclass.name)

                elif type(superclass) == Restriction:
                    CanonicalModel.concept_canonical_interpretation[concept].append('exists_' + superclass.property.name + '.' + superclass.value.name)

                elif type(superclass) == And:
                    if 'And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)) in CanonicalModel.concept_canonical_interpretation[concept]:
                        pass
                    else:
                        CanonicalModel.concept_canonical_interpretation[concept].append('And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)))

    def get_role_name_caninterp(self):

        # First case from Definition 10

        for role_name in self.role_names_iter:

            role_name_str = role_name.name # Accesses the property type object's name as a string
            CanonicalModel.role_canonical_interpretation[role_name_str] = []

            for restriction_name in self.concept_restrictions_dict.keys(): # Where restriction_name denotes a \exists r.B type of concept 'exists_' + self.concept.property.name + '.' + self.concept.value.name
                c_B = self.concept_restrictions_dict[restriction_name].concept.value.name

                if role_name_str in restriction_name:
                    superclasses = self.domain[restriction_name].concept.ancestors(include_self=True, include_constructs=False)

                    for superclass in superclasses:
                        super_superclasses = superclass.ancestors(include_self=True, include_constructs=True)

                        for super_superclass in super_superclasses:

                            if type(super_superclass) == ThingClass:
                                c_D = super_superclass.name
                                CanonicalModel.role_canonical_interpretation[role_name_str].append(tuple(c_D, c_B))

                            elif type(super_superclass) == Restriction:
                                c_D = 'exists_' + super_superclass.property.name + '.' + super_superclass.value.name
                                CanonicalModel.role_canonical_interpretation[role_name_str].append(tuple(c_D, c_B))

                            elif type(super_superclass) == And:
                                c_D = 'And_' + ''.join(sorted(super_superclass.Classes[0].name + super_superclass.Classes[1].name))
                                CanonicalModel.role_canonical_interpretation[role_name_str].append(tuple(c_D, c_B))

        # Second case from Definition 10: r \sqsubset s 
            
        for role_name in self.role_names_iter:

            superroles = role_name.ancestors(include_self=True)
            role_name_str = role_name.name
            
            for superrole in superroles:
                for restriction_name in self.concept_restrictions_dict.keys():
                    if superrole.name in restriction_name:
                        pair = tuple((restriction_name, self.domain[restriction_name].concept.value.name))
                        CanonicalModel.role_canonical_interpretation[role_name_str].append(pair)

In [ ]:
'''
Main function for creating the canonical model.

    Args:
        onto_dir (str): a string pointing to the directory where the ontology is stored.

    Returns:
        canmodel (CanonicalModel): returns a variable containing the canonical model. 
        
        Attention: the interpretations of concept names and role names can also be accessed via class variables
        from the CanonicalModel class.
'''

def create_canonical_model(onto_dir):

    onto = get_ontology(onto_dir)
    onto = onto.load()
    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())
    individuals_iter = list(onto.individuals())

    #for concept in concept_names_iter:
    #    concept.namespace = onto.ontology

    #for role in role_names_iter:
    #    role.namespace = onto.ontology

    get_canonical_model_elements(concept_names_iter, role_names_iter, onto)

    print('')
    print('============================================================================')
    print('')
    print('Starting to reason.')
    print('')

    with onto:
        sync_reasoner()
        
    #onto.save("inferences_goslimyeast.owl")

    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())
    individuals_iter = list(onto.individuals())

    print('')
    print('============================================================================')
    print('')
    print('Done reasoning. Creating the canonical model.')
    print('')
    canmodel = CanonicalModel(CanonicalModelElements.concept_names, CanonicalModelElements.concept_intersections, CanonicalModelElements.concept_restrictions, CanonicalModelElements.all_concepts, role_names_iter)
    print('============================================================================')
    print('')
    print('Concluded creating canonical model.')

    return canmodel

In [ ]:
# Instantiates the canonical model

canmodel = create_canonical_model(dir)

In [ ]:
'''
Utility functions for initializing
the class EntityEmbedding. They
allow us to access dictionaries
containing indexes and canonical
interpretation of concepts
and roles as class.
'''

def get_concept_names_idx_dict(canmodel):
   conceptnames_idx_dict = {concept_name: idx for idx, concept_name in enumerate(CanonicalModel.concept_canonical_interpretation.keys())}
   return conceptnames_idx_dict

def get_role_names_idx_dict(canmodel):
    rolenames_idx_dict = {role_name: idx for idx, role_name in enumerate(CanonicalModel.role_canonical_interpretation.keys())}
    return rolenames_idx_dict

def get_entities_idx_dict(canmodel):
    entities_idx_dict = {entity: idx for idx, entity in enumerate(canmodel.domain.keys())}
    return entities_idx_dict

def get_domain_dict(canmodel):
    return canmodel.domain

# Atenção: a função mu está com complexidade alta devido aos for loops

In [ ]:
''' 
Class for obtaining the positional 
embedding for each entity in the domain
of the canonical interpretation.
It represents the Mu Function from the
paper.
'''

class EntityEmbedding:

    # Dictionaries for storing the indices of concept names and role names, entities pairs, respectively
    # Keys are strings and values are integers
    
    concept_names_idx_dict = get_concept_names_idx_dict(canmodel)
    role_names_idx_dict = get_role_names_idx_dict(canmodel)
    entities_idx_dict = get_entities_idx_dict(canmodel)

    # Dictionaries accessing the canonical interpretation of concepts and roles
    # Keys and values are strings
    
    concept_canonical_interpretation_dict = CanonicalModel.concept_canonical_interpretation
    role_canonical_interpretation_dict = CanonicalModel.role_canonical_interpretation

    # Dictionary storing the domain of the canonical model being embedded
    # IMPORTANT: Keys are strings and values are CanonicalModelElements type objects
    
    domain_dict = get_domain_dict(canmodel)

    # Dictionary for easy access to entity embeddings
    # It is initialized with empty values, iteratively built by the .get_embedding_vector() method
    # Key (str): Domain Entity / Value (np.array): EntityEmbedding.embedding_vector

    entity_entityvector_dict = dict.fromkeys(domain_dict.keys())

    def __init__(self, entity_name, emb_dim):
        self.name = entity_name
        self.emb_dim = emb_dim
        self.in_interpretation_of = []
        self.embedding_vector = self.get_embedding_vector()

    def get_embedding_vector(self):
        
        embedding_vector = np.zeros((self.emb_dim,))
        EntityEmbedding.entity_entityvector_dict[self.name] = []

        # Applies the embedding function to the concept names portion of the definition

        for concept_name in EntityEmbedding.concept_canonical_interpretation_dict:
            concept_name_idx = EntityEmbedding.concept_names_idx_dict[concept_name]
        
            if self.name in EntityEmbedding.concept_canonical_interpretation_dict[concept_name]:
                embedding_vector[concept_name_idx] = 1
                self.in_interpretation_of.append(concept_name)

        # Applies the embedding function to the role names portion of the definition

        for role_name in EntityEmbedding.role_canonical_interpretation_dict:
            
            role_name_idx = len(EntityEmbedding.concept_names_idx_dict) + (EntityEmbedding.role_names_idx_dict[role_name] * len(EntityEmbedding.entities_idx_dict))
            role_name_caninterp = EntityEmbedding.role_canonical_interpretation_dict[role_name]

            for pair in role_name_caninterp:

                entity_2 = pair[1]

                if (self.name, entity_2) == pair:
                    entity_2_idx = EntityEmbedding.entities_idx_dict[entity_2]
                    final_role_entity_pair_idx = role_name_idx + entity_2_idx
                    embedding_vector[final_role_entity_pair_idx] = 1

        # EntityEmbedding.entity_entityvector_dict[self.name].append(embedding_vector)
        EntityEmbedding.entity_entityvector_dict[self.name] = embedding_vector

        return embedding_vector

In [ ]:
'''
Function for creating the binary vectors representing
each element of the domain of the canonical interpretation.

    Args:
        emb_dim (int/float): the number of dimensions of the embedding space.

    Returns:
        embedded_entities (list): a list containing all embeddings of the entities
                                  in the domain. 
    
    The embedded_entities are also available in the dictionary EntityEmbeddings.entity_entityvector_dict
'''

def get_domain_embeddings(emb_dim):

    embedded_entities = []
    counter = 0

   # The entities in the domain are strings
    
    for entity_name in EntityEmbedding.domain_dict:
       embedded_entity = EntityEmbedding(entity_name, emb_dim)
       embedded_entities.append(embedded_entity)
       counter += 1
       
       if counter % 1000 == 0:
           print(counter)
       
    return embedded_entities

In [ ]:
'''
Final class for creating the dataset.

Inputs: concept or role names, generated
embeddings for entities in the domain of
the canonical model.

Outputs: geometrical interpretation of
concepts and role names, represented
by vertices defining a region.

One can access the GeometricInterpretation
objects either as elements in a list, or as
values in a class variable dictionary.
'''

class GeometricInterpretation:

    concept_geointerps_dict = dict.fromkeys(CanonicalModel.concept_canonical_interpretation.keys())
    role_geointerps_dict = dict.fromkeys(CanonicalModel.role_canonical_interpretation.keys())

    def __init__(self, name, emb_dim):
        self.name = name
        self.emb_dim = emb_dim
        self.vertices = []
        self.centroid = None
        self.bounding_box_vertices = None

    def get_centroid_naive(self):
        if len(self.vertices) == 0 and self.name in self.concept_geointerps_dict.keys():
            centroid = np.zeros((self.emb_dim,))
            return centroid
        
        elif len(self.vertices) == 0 and self.name in self.role_geointerps_dict.keys():
            centroid = np.zeros((self.emb_dim * 2,)) # The centroid for the regions needs to be doubled due to the concat operation
            return centroid
        
        elif len(self.vertices) > 0 and self.name in self.concept_geointerps_dict.keys():
            n = len(self.vertices)
            centroid = np.zeros((self.emb_dim,))
            matrix = np.vstack(self.vertices)
            centroid = 1/n * np.sum(matrix, axis=0)
            return centroid
        
        elif len(self.vertices) > 0 and self.name in self.role_geointerps_dict.keys():
            n = len(self.vertices)
            centroid = np.zeros((self.emb_dim,))
            matrix = np.vstack(self.vertices)
            centroid = 1/n * np.sum(matrix, axis=0)
            return centroid

    def get_bounding_box_vertices(self):
        pass
        

# There has to be a more efficient way of doing the creating of geometric interpretations for concepts and roles

In [ ]:
def index_finder(emb_dim, concept_name_idx_dict, role_name_idx_dict, domain_idx_dict):

    index_dict = {k: None for k in range(emb_dim)}

    for k,v in concept_name_idx_dict.items():

        index_dict[v] = k

    for role in role_name_idx_dict:
        role_init_idx = len(concept_name_idx_dict) + (role_name_idx_dict[role] * len(domain_idx_dict))

        for entity in domain_idx_dict:
            entity_init_idx = domain_idx_dict[entity]
            final_role_entity_pair_idx = role_init_idx + entity_init_idx
            index_dict[final_role_entity_pair_idx] = (role, entity)
        
    return index_dict

In [ ]:
def get_faithful_concept_geometric_interps(concept_names_interps, domain_embeddings_list, entity_dims_index_dict, emb_dim, canmodel: CanonicalModel):

    faithful_concept_geometric_interps = []

    for concept_name in concept_names_interps.keys():
        concept_name = GeometricInterpretation(concept_name, emb_dim)

        for embedding in domain_embeddings_list:
            if concept_name.name in embedding.in_interpretation_of:
                concept_name.vertices.append(embedding.embedding_vector)
            
        GeometricInterpretation.concept_geointerps_dict[concept_name.name] = concept_name
        concept_name.centroid = concept_name.get_centroid_naive()
        
        faithful_concept_geometric_interps.append(concept_name)

    return faithful_concept_geometric_interps

In [ ]:
def get_faithful_role_geometric_interps(role_names_interps, entity_embeddings_list, entity_dims_index_dict, emb_dim, canmodel: CanonicalModel):
    
    faithful_role_geometric_interps = []
    idx_entity_dict = entity_dims_index_dict
    #entity_idx_dict = {v: k for k,v in entity_dims_index_dict}

    relevant_idxs = len(canmodel.concept_names_dict)-1

    for role_name in role_names_interps.keys():
        role_name_str = role_name
        role_name = GeometricInterpretation(role_name_str, emb_dim)

        for entity in entity_embeddings_list:

            onehot_idx_list = np.where(entity.embedding_vector == 1)[0]
            #print(f'This is the entity: {entity} and this is the onehot_idx_list: {onehot_idx_list}')

            for idx in onehot_idx_list: # I could just look at the TRULY relevant indexes
                if idx > relevant_idxs:
                    role_entity_pair = idx_entity_dict[idx]
                    r_name_str = role_entity_pair[0]
                    e_name_str = role_entity_pair[1]

                    if r_name_str == role_name_str:
                        e_embedding = EntityEmbedding.entity_entityvector_dict[e_name_str]
                        role_name.vertices.append(np.concatenate((entity.embedding_vector, e_embedding)))

        GeometricInterpretation.role_geointerps_dict[role_name_str] = role_name
        role_name.centroid = role_name.get_centroid_naive()
        faithful_role_geometric_interps.append(role_name)

    return faithful_role_geometric_interps

In [ ]:
def create_tbox_embeddings(canonical_model: CanonicalModel):

    domain = canonical_model.domain # Keys are strings and values are CanonicalModelElements type objects
    concept_names_interps = canonical_model.concept_canonical_interpretation # Keys are strings and values are lists of strings.
    role_names_interps = canonical_model.role_canonical_interpretation # Keys are strings and values are lists of tuples. Tuples are of form ('C', 'D'), with C and D strings.

    EMB_DIM = len(concept_names_interps) + len(role_names_interps) * len(domain)

    print('================EMBEDDING DIMENSION================')
    print(f'Concept Name dimensions: {len(concept_names_interps)}')
    print(f'The number of role names is: {len(role_names_interps)}')
    print(f'The size of the domain is: {len(domain)}')
    print(f'Role names dimensions: {len(role_names_interps) * len(domain)}')
    print('===================================================')
    print('')
    print(f'Final embedding dimension: {EMB_DIM}')
    print(f'The final dimension for role regions is: {EMB_DIM * 2}')

    domain_embeddings_list = get_domain_embeddings(EMB_DIM)
    
    concept_names_ordering = EntityEmbedding.concept_names_idx_dict
    role_names_ordering = EntityEmbedding.role_names_idx_dict
    entities_ordering = EntityEmbedding.entities_idx_dict
    
    print('')
    print('===============FINISHED EMBEDDINGS===============')
    print(f'There are {len(domain_embeddings_list)} vector embeddings.')
    print('')

    index_finder_dict = index_finder(EMB_DIM, concept_names_ordering, role_names_ordering, entities_ordering)

    faithful_concept_geometric_interps = get_faithful_concept_geometric_interps(concept_names_interps, domain_embeddings_list, index_finder_dict, EMB_DIM, canonical_model)

    print('============FINISHED INTERPS CONCEPT=============')
    print(f'There are {len(faithful_concept_geometric_interps)} regions for concept names.')
    print('')

    faithful_role_geometric_interps = get_faithful_role_geometric_interps(role_names_interps, domain_embeddings_list, index_finder_dict, EMB_DIM, canonical_model)

    print('=============FINISHED INTERPS ROLES==============')
    print(f'There are {len(faithful_role_geometric_interps)} regions for role names.')
    print('')

    return domain_embeddings_list, faithful_concept_geometric_interps, faithful_role_geometric_interps, index_finder_dict # Returns the faithful geometric interpretations for concepts and roles as lists

In [ ]:
domain_embeddings, concept_geointerps, role_geointerps, idx_finder_dict = create_tbox_embeddings(canmodel)

In [ ]:
'''
Function for creating the pre-split dataset containing facts from the ontology.
Distinguishes between concept assertions and role assertions.


    Args: ontology_dir (str): the directory from the ontology
          concept_geointerps_dict (dict): the geometrical interpretations for concepts generated by create_tbox_embeddings()
          role_geointerps_dict (dict): the geometrical interpretations for roles generated by create_tbox_embeddings()

    Returns:
          X_concepts (np.array): A dataframe with columns 'Concept', 'Entity', 'y_true' (equivalent to concept.centroid())
          X_roles (np.array): A dataframe with columns 'SubjectEntity', 'Role', 'ObjectEntity', 'y_true' (equivalent to role.centroid())
          y_concepts (np.array):
          y_roles (np.array):
          vocabulary_dict (dict): A vocabulary with key (int): value (str) for entities in the domain.
'''

def get_abox_dataset(ontology_dir: str, concept_geointerps_dict: dict, role_geointerps_dict: dict, concept_to_idx: dict, role_to_idx: dict):
    
    ontology = get_ontology(ontology_dir)
    ontology = ontology.load()
    
    X_concepts = []
    X_roles = []
    y_concepts = []
    y_roles = []

    entities = [entity.name for entity in list(ontology.individuals())]
    
    concept_to_idx_vocab = concept_to_idx
    idx_to_concept_vocab = {value: key for key, value in concept_to_idx_vocab.items()}

    role_to_idx_vocab = role_to_idx
    idx_to_role_vocab = {value: key for key, value in role_to_idx_vocab.items()}
    
    entity_to_idx_vocab = {value: index for index, value in enumerate(entities)}
    idx_to_entity_vocab = {value: key for key, value in entity_to_idx_vocab.items()}

    for individual in list(ontology.individuals()):

        # all_facts = individual.INDIRECT_is_a # Is this actually getting all assertions?
        all_facts = individual.is_a

        for concept in all_facts:
            if type(concept) == ThingClass:
                concept = concept_geointerps_dict[concept.name]
                fact = np.array([concept_to_idx_vocab[concept.name], entity_to_idx_vocab[individual.name]])
                y_label = np.array(concept.centroid)
                X_concepts.append(fact)
                y_concepts.append(y_label)
                
            elif type(concept) == And:
                print('And') # remove print statement
                concept1 = concept_geointerps_dict[concept.Classes[0].name]
                concept2 = concept_geointerps_dict[concept.Classes[1].name]
                concept_name = 'And_' + ''.join(sorted(concept1.name + concept2.name))
                fact = np.array([concept_to_idx_vocab[concept_name], entity_to_idx_vocab[individual.name]])
                y_label = np.array((concept1.centroid + concept2.centroid)/2) # The golden label for an intersection is just the average of the centroid of the two regions
                X_concepts.append(fact)
                y_concepts.append(y_label)
            
            elif type(concept) == Restriction:
                print('restriction') # remove print statement
                concept = concept_geointerps_dict['exists_' + concept.property.name + '.' + concept.value.name]
                fact = np.array([concept_to_idx_vocab[concept.name], entity_to_idx_vocab[individual.name]])
                y_label = np.array(concept.centroid)
                X_concepts.append(fact)
                y_concepts.append(y_label)

        relevant_roles = individual.get_properties()
        individual_name = individual.name

        for role in relevant_roles:
            role_geo = role_geointerps_dict[role.name]
            subject_list = role[individual] # This syntax is from the owlready2 library
            for subject in subject_list:
                fact = np.array([entity_to_idx_vocab[individual.name], role_to_idx_vocab[role.name], entity_to_idx_vocab[subject.name]])
                X_roles.append(fact)
                y_label = y_roles.append(np.array(role_geo.centroid))

    return np.array(X_concepts), np.array(X_roles), np.array(y_concepts), np.array(y_roles), entity_to_idx_vocab, idx_to_entity_vocab, concept_to_idx_vocab, idx_to_concept_vocab, role_to_idx_vocab, idx_to_role_vocab

In [ ]:
X_concepts, X_roles, y_concepts, y_roles, entity_to_idx_vocab, idx_to_entity_vocab, concept_to_idx_vocab, idx_to_concept_vocab, role_to_idx_vocab, idx_to_role_vocab = get_abox_dataset(dir,
                                                                                                                                                                                        GeometricInterpretation.concept_geointerps_dict,
                                                                                                                                                                                        GeometricInterpretation.role_geointerps_dict,
                                                                                                                                                                                        EntityEmbedding.concept_names_idx_dict,
                                                                                                                                                                                        EntityEmbedding.role_names_idx_dict)

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.data import Subset

In [ ]:
class OntologyDataset(Dataset):
    def __init__(self, data, labels):
        self.X = torch.tensor(data, dtype=torch.long)
        self.y = torch.tensor(labels, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].long(), self.y[idx]

In [ ]:
TRAIN_SIZE_PROPORTION = 0.8

In [ ]:
torch.manual_seed(100)
ConceptDataset = OntologyDataset(X_concepts, y_concepts)

dataset_size = len(ConceptDataset)
train_size = int(TRAIN_SIZE_PROPORTION * dataset_size)
test_size = dataset_size - train_size

trainConceptDataset, testConceptDataset = torch.utils.data.random_split(ConceptDataset, [train_size, test_size])

In [ ]:
torch.manual_seed(100)
RoleDataset = OntologyDataset(X_roles, y_roles)

dataset_size = len(RoleDataset)
train_size = int(TRAIN_SIZE_PROPORTION * dataset_size)
test_size = dataset_size - train_size

trainRoleDataset, testRoleDataset = torch.utils.data.random_split(RoleDataset, [train_size, test_size])

In [ ]:
torch.manual_seed(100)
BATCH_SIZE = 32

train_ConceptDataLoader = DataLoader(trainConceptDataset, batch_size = BATCH_SIZE, shuffle=False)
test_ConceptDataLoader = DataLoader(testConceptDataset, batch_size = BATCH_SIZE, shuffle=False)

train_RoleDataLoader = DataLoader(trainRoleDataset, batch_size = BATCH_SIZE, shuffle=False)
test_RoleDataLoader = DataLoader(testRoleDataset, batch_size = BATCH_SIZE, shuffle=False)